<a href="https://colab.research.google.com/github/bahram3000/test_balancer_app/blob/main/clear_deposit_rebalace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qqq web3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 63.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.5/587.5 kB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.5/102.5 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 340.3/340.3 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.8/175.8 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 93.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 91.4 MB/s eta 0:00:00


In [80]:
from web3 import Web3
from eth_account import Account
import requests
import json
import time
import decimal
import numpy as np
import math

In [4]:
RPC = "https://eth-sepolia.g.alchemy.com/v2/demo"
w3 = Web3(Web3.HTTPProvider(RPC))

# کلید خصوصی والت تست‌نت
PRIVATE_KEY = "75074ee5760d9640d993d09bc6e969b37cadcc2c83195886d43b3e86b2b8b6da"
account = Account.from_key(PRIVATE_KEY)
address = account.address

print("Connected:", w3.is_connected())
print("Your address:", address)

Connected: True
Your address: 0x7868EDEfC36A74eba220C75aCFB05c59a4e55822


In [5]:
def find_key(data, target_key):
    results = []
    if isinstance(data, dict):
        for key, value in data.items():
            if key == target_key:
                results.append(value)
            else:
                results.extend(find_key(value, target_key))
    elif isinstance(data, list):
        for item in data:
            results.extend(find_key(item, target_key))
    return results

In [6]:
def check_token_balance(token_list_address:list, inp_address):
  ERC20_ABI = '''
  [
  {"constant":true,"inputs":[{"name":"_owner","type":"address"}],
   "name":"balanceOf","outputs":[{"name":"balance","type":"uint256"}],
   "type":"function"},
  {"constant":true,"inputs":[],"name":"decimals",
   "outputs":[{"name":"","type":"uint8"}],"type":"function"},
  {"constant":true,"inputs":[],"name":"symbol",
   "outputs":[{"name":"","type":"string"}],"type":"function"}
  ]
  '''
  balances = []
  for token in token_list_address:
      contract = w3.eth.contract(address=token, abi=ERC20_ABI)
      bal = contract.functions.balanceOf(inp_address).call()
      dec = contract.functions.decimals().call()
      sym = contract.functions.symbol().call()
      balances.append([sym, bal / (10 ** dec)])
  eth_balance = w3.eth.get_balance(address)
  balances.append(["ETH", w3.from_wei(eth_balance, 'ether')])
  return balances

In [14]:
etherscan_apikey = "N68HP1FGB4UQQFBHZQUJIM12HVYWJME96M"
CHAIN_ID = 11155111  # Sepolia

In [15]:
weth_address="0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14"
usdc_address="0x1c7D4B196Cb0C7B01d743Fbc6116a902379C7238"
uni_address="0x1f9840a85d5aF5bf1D1762F925BDADdC4201F984"
my_wallet_address = "0x7868EDEfC36A74eba220C75aCFB05c59a4e55822"
eth_usdc_pool_address="0x3289680dD4d6C10bb19b899729cda5eEF58AEfF1"

In [13]:
TOKEN_LIST = [weth_address,usdc_address,uni_address]

In [16]:
weth_address=w3.to_checksum_address(weth_address)
usdc_address=w3.to_checksum_address(usdc_address)
uni_address=w3.to_checksum_address(uni_address)
my_wallet_address=w3.to_checksum_address(my_wallet_address)
eth_usdc_pool_address=w3.to_checksum_address(eth_usdc_pool_address)

In [17]:
def fetch_abi(contract_address: str, api_key: str,inp_chain_id:int):
    url = "https://api.etherscan.io/v2/api"
    params = {
        "module": "contract",
        "action": "getabi",
        "address": contract_address,
        "apikey": api_key,
        "chainid":inp_chain_id
    }
    resp = requests.get(url, params=params)
    resp.raise_for_status()
    data = resp.json()
    if data.get("status") != "1":
        raise Exception(f"Error getting ABI: {data.get('message')}")
    abi = json.loads(data["result"])
    return abi

In [19]:
ERC20_ABI = fetch_abi(weth_address,etherscan_apikey,CHAIN_ID)
POOL_ABI = fetch_abi(eth_usdc_pool_address,etherscan_apikey,CHAIN_ID)

In [20]:
weth_contract = w3.eth.contract(address=weth_address, abi=ERC20_ABI)
usdc_contract = w3.eth.contract(address=usdc_address, abi=ERC20_ABI)
pool_contract = w3.eth.contract(address=eth_usdc_pool_address, abi=POOL_ABI)

In [38]:
def pool_current_info(inp_pool_contract):
  token0_address = inp_pool_contract.functions.token0().call()
  token1_address = inp_pool_contract.functions.token1().call()
  slot0 = inp_pool_contract.functions.slot0().call()
  liquidity = inp_pool_contract.functions.liquidity().call()
  fee = inp_pool_contract.functions.fee().call()
  tickspacing = inp_pool_contract.functions.tickSpacing().call()
  return {'token0_address':token0_address,'token1_address':token1_address,'sqrtprice':slot0[0],'current_tick':slot0[1],'liquidity':liquidity,'fee_tier':fee,'tick_spacing':tickspacing}

In [39]:
pool_info=pool_current_info(pool_contract)
pool_info

{'token0_address': '0x1c7D4B196Cb0C7B01d743Fbc6116a902379C7238',
 'token1_address': '0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14',
 'sqrtprice': 881750197598185948484022449845594,
 'current_tick': 186355,
 'liquidity': 11476736666502393,
 'fee_tier': 500,
 'tick_spacing': 10}

In [25]:
def calculate_price_by_tick(tick:int,decimal_0:int,decimal_1:int):
  return (10**(decimal_0-decimal_1))*(1.0001**tick)

In [26]:
def calculate_price_from_sqrt_price(sqrt_price_raw: float, decimal_0: int, decimal_1: int) -> float:
    normalized_sqrt_price = sqrt_price_raw / (2 ** 96)
    raw_price_math = normalized_sqrt_price ** 2
    decimal_adjustment = decimal_0 - decimal_1
    final_price = raw_price_math * (10 ** decimal_adjustment)
    return final_price

In [30]:
token0_contract = w3.eth.contract(address=pool_info['token0_address'], abi=ERC20_ABI)
token1_contract = w3.eth.contract(address=pool_info['token1_address'], abi=ERC20_ABI)

In [31]:
calculate_price_by_tick(pool_info['current_tick'],token0_contract.functions.decimals().call(),token1_contract.functions.decimals().call())

0.00012384833710346743

In [35]:
calculate_price_from_sqrt_price(pool_info['sqrtprice'],token0_contract.functions.decimals().call(),token1_contract.functions.decimals().call())

0.00012386025354469175

In [34]:
#eth_price = requests.get("https://api.coingecko.com/api/v3/simple/price?ids=ethereum&vs_currencies=usd").json()["ethereum"]["usd"]
eth_sepolia_price=1/calculate_price_from_sqrt_price(pool_info['sqrtprice'],token0_contract.functions.decimals().call(),token1_contract.functions.decimals().call())
eth_sepolia_price

8073.614992554299

In [37]:
POSITION_MANAGER_ADDRESS = "0x1238536071E1c677A632429e3655c799b22cDA52"
POSITION_MANAGER_ADDRESS = w3.to_checksum_address(POSITION_MANAGER_ADDRESS)
POSITION_MANAGER_ABI = fetch_abi(POSITION_MANAGER_ADDRESS,etherscan_apikey,CHAIN_ID)
POSITION_MANAGER_contract = w3.eth.contract(address=POSITION_MANAGER_ADDRESS, abi=POSITION_MANAGER_ABI)

In [40]:
def check_liquidity_nft(inp_address,inp_postion_manger_address,inp_position_manager_abi):
  pm_contract=w3.eth.contract(address=inp_postion_manger_address, abi=inp_position_manager_abi)
  nfts=[]
  try:
    balance = pm_contract.functions.balanceOf(address).call()
    print(f"تعداد NFT پوزیشن‌ها: {balance}")


    if balance > 0:
        for i in range(balance):

            token_id = pm_contract.functions.tokenOfOwnerByIndex(address, i).call()

            pos = pm_contract.functions.positions(token_id).call()
            dct={'token_id':token_id,'token0':pos[0],'token1':pos[3],'fee_tier':pos[4],'liquidity':pos[7],'tick_range':[pos[5],pos[6]]}
            nfts.append(dct)
        return nfts
    else:
        print("lp nft not found")

  except Exception as e:
    print(f"error in read data --> {e}")

In [42]:
check_liquidity_nft(my_wallet_address,POSITION_MANAGER_ADDRESS,POSITION_MANAGER_ABI)

تعداد NFT پوزیشن‌ها: 4


[{'token_id': 217011,
  'token0': 0,
  'token1': '0x1f9840a85d5aF5bf1D1762F925BDADdC4201F984',
  'fee_tier': 500,
  'liquidity': 0,
  'tick_range': [142500, 156410]},
 {'token_id': 220887,
  'token0': 0,
  'token1': '0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14',
  'fee_tier': 10000,
  'liquidity': 1824073543371,
  'tick_range': [184800, 186800]},
 {'token_id': 220890,
  'token0': 0,
  'token1': '0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14',
  'fee_tier': 10000,
  'liquidity': 15531994169743,
  'tick_range': [185000, 187000]},
 {'token_id': 220931,
  'token0': 0,
  'token1': '0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14',
  'fee_tier': 500,
  'liquidity': 1581033524365,
  'tick_range': [185170, 187400]}]

In [43]:
v3factoryaddress="0x0227628f3F023bb0B980b67D528571c95c6DaC1c"

In [46]:
def get_v3_pool_address(inp_token0_address,inp_token1_address,inp_fee_tier,inp_factory_address,inp_scan_apikey,inp_chainid):
  FACTORY_ADDRESS = w3.to_checksum_address(v3factoryaddress)
  FACTORY_ABI =fetch_abi(inp_factory_address,etherscan_apikey,CHAIN_ID)
  factory = w3.eth.contract(address=FACTORY_ADDRESS, abi=FACTORY_ABI)
  pool_address_v = factory.functions.getPool(inp_token0_address, inp_token1_address, inp_fee_tier).call()
  return pool_address_v

In [47]:
get_v3_pool_address(usdc_address,weth_address,500,v3factoryaddress,etherscan_apikey,CHAIN_ID)

'0x3289680dD4d6C10bb19b899729cda5eEF58AEfF1'

In [ ]:
#token0:usdc,token1:weth

In [78]:
def get_liquidity_for_amount0(amount0, sqrt_p, sqrt_ph):
    if sqrt_p >= sqrt_ph: return 0
    return amount0 / ((1 / sqrt_p) - (1 / sqrt_ph))

def get_liquidity_for_amount1(amount1, sqrt_p, sqrt_pl):
    if sqrt_p <= sqrt_pl: return 0
    return amount1 / (sqrt_p - sqrt_pl)

def calculate_other_token_amount(input_amount, is_token0_input, current_price, price_low, price_high):
    sqrt_p = math.sqrt(current_price)
    sqrt_pl = math.sqrt(price_low)
    sqrt_ph = math.sqrt(price_high)

    liquidity = 0.0
    if price_low < current_price < price_high:
        if is_token0_input:
            liquidity = get_liquidity_for_amount0(input_amount, sqrt_p, sqrt_ph)
        else:
            liquidity = get_liquidity_for_amount1(input_amount, sqrt_p, sqrt_pl)

    elif current_price <= price_low:
        if is_token0_input:
            liquidity = input_amount / ((1/sqrt_pl) - (1/sqrt_ph))
        else:
            return 0.0, 0.0, 0.0
    elif current_price >= price_high:
        if not is_token0_input:
            liquidity = input_amount / (sqrt_ph - sqrt_pl)
        else:
            return 0.0, 0.0, 0.0

    final_amount0 = 0.0
    final_amount1 = 0.0

    if price_low < current_price < price_high:
        final_amount0 = liquidity * ((1/sqrt_p) - (1/sqrt_ph))
        final_amount1 = liquidity * (sqrt_p - sqrt_pl)


    elif current_price <= price_low:
        final_amount0 = liquidity * ((1/sqrt_pl) - (1/sqrt_ph))
        final_amount1 = 0

    elif current_price >= price_high:
        final_amount0 = 0
        final_amount1 = liquidity * (sqrt_ph - sqrt_pl)

    return final_amount0, final_amount1, liquidity

In [81]:
def calculate_amounts(current_price, price_low, price_high, target_liquidity):

    sqrt_p = math.sqrt(current_price)
    sqrt_pl = math.sqrt(price_low)
    sqrt_ph = math.sqrt(price_high)

    amount0 = 0.0
    amount1 = 0.0

    if current_price <= price_low:
        amount0 = target_liquidity * ((1 / sqrt_pl) - (1 / sqrt_ph))
        amount1 = 0.0

    elif current_price >= price_high:
        amount0 = 0.0
        amount1 = target_liquidity * (sqrt_ph - sqrt_pl)

    else:
        amount0 = target_liquidity * ((1 / sqrt_p) - (1 / sqrt_ph))
        amount1 = target_liquidity * (sqrt_p - sqrt_pl)

    return amount0, amount1

In [83]:
calculate_other_token_amount(100,False,eth_sepolia_price,eth_sepolia_price*0.8,eth_sepolia_price*1.2)

(0.01022216696294876, 100.0, 10.541778949606618)

In [51]:
decimal_0=token0_contract.functions.decimals().call()
decimal_1=token1_contract.functions.decimals().call()

In [ ]:
approve_tx0 = token0_contract.functions.approve(POSITION_MANAGER_ADDRESS, 2**256-1).build_transaction({
    "from": my_wallet_address,
    "nonce": w3.eth.get_transaction_count(my_wallet_address)+1
})
# sign + send

approve_tx1 = token1_contract.functions.approve(POSITION_MANAGER_ADDRESS, 2**256-1).build_transaction({
    "from": my_wallet_address,
    "nonce": w3.eth.get_transaction_count(my_wallet_address)+1
})
# sign + send
print(approve_tx0)
print(approve_tx1)

In [53]:
portfolio=check_token_balance(TOKEN_LIST,my_wallet_address)
portfolio

[['WETH', 0.03764500076879961],
 ['USDC', 64.902607],
 ['UNI', 0.000830846866961009],
 ['ETH', Decimal('0.155721429076476146')]]

In [67]:
init_amount1=portfolio[0][1]/10
init_amount0=init_amount1*eth_sepolia_price
print(init_amount0)
print(init_amount1)

30.393124260169866
0.003764500076879961


In [77]:
init_calculate_Liquid_amounts((1/eth_sepolia_price)*(0.8),(1/eth_sepolia_price)*(1.2),2*init_amount1)

(np.float64(3.2039759753863715), 30.393124260169863, 0.003764500076879961)

In [76]:
calculate_new_amounts((1/eth_sepolia_price),(1/eth_sepolia_price)*(0.8),(1/eth_sepolia_price)*(1.2),2*init_amount1)

(np.float64(25.083396959219332), np.float64(0.003764500076879961))

In [ ]:
# مقادیر درخواستی شما در تابع mint
req_amount0 = 85584095
req_amount1 = 10000000000000000

print(f"--- Checking Token 0 ---")
balance0 = token0_contract.functions.balanceOf(my_address).call()
allowance0 = token0_contract.functions.allowance(my_address, POSITION_MANAGER_ADDRESS).call()

print(f"Required:  {req_amount0}")
print(f"Balance:   {balance0}   -> {'✅ OK' if balance0 >= req_amount0 else '❌ INSUFFICIENT BALANCE'}")
print(f"Allowance: {allowance0} -> {'✅ OK' if allowance0 >= req_amount0 else '❌ NOT APPROVED YET'}")

print(f"\n--- Checking Token 1 ---")
balance1 = token1_contract.functions.balanceOf(my_address).call()
allowance1 = token1_contract.functions.allowance(my_address, POSITION_MANAGER_ADDRESS).call()

print(f"Required:  {req_amount1}")
print(f"Balance:   {balance1}   -> {'✅ OK' if balance1 >= req_amount1 else '❌ INSUFFICIENT BALANCE'}")
print(f"Allowance: {allowance1} -> {'✅ OK' if allowance1 >= req_amount1 else '❌ NOT APPROVED YET'}")

In [ ]:
amount_to_approve = 2**256 - 1  # Infinite approval

print("🚀 Sending Approve Transaction for Token 1...")

# 1. گرفتن Nonce دقیق (شامل تراکنش‌های پندینگ)
nonce = w3.eth.get_transaction_count(my_address, 'pending')

# 2. ساخت تراکنش
tx = token1_contract.functions.approve(POSITION_MANAGER_ADDRESS, amount_to_approve).build_transaction({
    'from': my_address,
    'nonce': nonce,
    'gasPrice': w3.eth.gas_price  # اختیاری: برای شبکه هایی مثل پالیگان لازم است
})

# 3. امضا و ارسال
signed_tx = w3.eth.account.sign_transaction(tx, private_key=PRIVATE_KEY)
tx_hash = w3.eth.send_raw_transaction(signed_tx.raw_transaction)

print(f"⏳ Transaction sent! Hash: {w3.to_hex(tx_hash)}")
print("Waiting for confirmation...")

# 4. صبر برای تایید (مهمترین بخش)
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

if tx_receipt.status == 1:
    print("✅ Token 1 Approved Successfully!")
    print("👉 Now you can run your MINT function.")
else:
    print("❌ Approve Transaction Failed on-chain.")

In [ ]:
pm_contract = w3.eth.contract(address=POSITION_MANAGER_ADDRESS, abi=MINT_ABI)

params = {
    "token0": token0_addr,
    "token1": token1_addr,
    "fee":10000,
    "tickLower": 185000,
    "tickUpper": 187000,
    "amount0Desired" : int(amount_usdc * 10**6),       # واحد USDC
    "amount1Desired" : w3.to_wei(amount_weth, "ether"),  # واحد WETH
    "amount0Min": 0,
    "amount1Min": 0,
    "recipient": my_address,
    "deadline": int(time.time()) + 1200
}

tx = pm_contract.functions.mint(params).build_transaction({
    "from": my_address,
    "nonce": w3.eth.get_transaction_count(my_address),
    "gas": 500000,
    "gasPrice": w3.to_wei("2", "gwei")
})

signed_tx = w3.eth.account.sign_transaction(tx, PRIVATE_KEY)
tx_hash = w3.eth.send_raw_transaction(signed_tx.raw_transaction)
try:
  print("TX SENT:", tx_hash.hex())
except:
  print(tx_hash)
